### Configuration of the environment and import of libraries

In [1]:
import sys
import os
import json

# Navigate up two directories to reach 'backend'
backend_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(backend_path)

# Now you can import modules from the 'backend' directory
from backend.crud.crud_knowledge_base import crud_knowledge_base
from sqlalchemy.orm import Session  # type: ignore
from sqlalchemy import create_engine  # type: ignore
from backend.core.config import settings

c:\Users\dongu\miniconda3\envs\sole_mate_ai\Lib\site-packages\pydantic\_internal\_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


In [2]:
engine = create_engine(
    f"postgresql://{settings.POSTGRES_USER}:{settings.POSTGRES_PASSWORD}@{settings.POSTGRES_SERVER}:{settings.POSTGRES_PORT}/{settings.POSTGRES_DB}"
)

Fetch shoes from database

In [3]:
shoes_kb = crud_knowledge_base.get_default_kn_chatbot(db=Session(engine), limit=25)
for shoe in shoes_kb:

    print(shoe.__dict__)

{'_shoe_id': UUID('216fba03-c388-4703-9d30-2c677d4d6abb'), '_shoe_name': 'Reebok Shoe Model Blue 42', '_brand_name': 'Reebok', '_size_number': 42, '_color_name': 'Blue', '_discounted_price': 1260000.0, '_promotion_name': 'Quay Trở Lại Trường Học', '_promotion_start_date': '01-09-2024', '_promotion_end_date': '30-09-2024', '_promotion_discount_percent': 10}
{'_shoe_id': UUID('2bf6873c-32e8-472e-915f-f0251c2679a7'), '_shoe_name': 'Reebok Shoe Model Blue 40', '_brand_name': 'Reebok', '_size_number': 40, '_color_name': 'Blue', '_discounted_price': 1350000.0, '_promotion_name': 'Quay Trở Lại Trường Học', '_promotion_start_date': '01-09-2024', '_promotion_end_date': '30-09-2024', '_promotion_discount_percent': 10}
{'_shoe_id': UUID('2da19042-3a33-4b4e-86fe-e43c616ec4fa'), '_shoe_name': 'Reebok Shoe Model Light Blue 40', '_brand_name': 'Reebok', '_size_number': 40, '_color_name': 'Light Blue', '_discounted_price': 1800000.0, '_promotion_name': 'Quay Trở Lại Trường Học', '_promotion_start_date

In [4]:
brand_name = [shoe.get_brand_name() for shoe in shoes_kb]
unique_brands = set(brand_name)
print(", ".join(unique_brands))

Nike, Reebok, Puma, Adidas


Mokup of the data and format

In [5]:
import os
import json


# Hàm tạo phản hồi và ngữ cảnh cho từng câu hỏi
def create_responses_and_context(shoes_kb):
    responses_and_context = {}
    brand_name = [shoe.get_brand_name() for shoe in shoes_kb]
    shoe_name = [shoe.get_shoe_name() for shoe in shoes_kb]
    size_number = [shoe.get_size_number() for shoe in shoes_kb]
    color_name = [shoe.get_color_name() for shoe in shoes_kb]
    # discounted_price = [shoe.get_discounted_price() for shoe in shoes_kb]
    promotion_name = [shoe.get_promotion_name() for shoe in shoes_kb]

    responses_and_context[f"Có tất cả bao nhiêu mặc hàng", "giày"] = {
        "response": f"Tổng số giày hiện có là {len(set(shoe_name))}. Danh sách các giày: {', '.join(map(str,set(shoe_name)))}.",
        "context": f"Thông tin về số lượng giày hiện có.",
    }
    responses_and_context[f"Có tất cả bao nhiêu thương hiệu giày", "thương hiệu"] = {
        "response": f"Tổng số thương hiệu giày hiện có là {len(set(brand_name))}. Danh sách các thương hiệu giày: {', '.join(map(str,set(brand_name)))}.",
        "context": f"Thông tin về số lượng thương hiệu giày hiện có.",
    }
    responses_and_context[f"Có tất cả bao nhiêu màu giày", "màu"] = {
        "response": f"Tổng số màu giày hiện có là {len(set(color_name))}. Danh sách các màu giày: {', '.join(map(str,set(color_name)))}.",
        "context": f"Thông tin về số lượng màu giày hiện có.",
    }
    responses_and_context[f"Có tất cả bao nhiêu kích thước giày", "size"] = {
        "response": f"Tổng số size giày hiện có là {len(set(size_number))}. Danh sách các size giày: {', '.join(map(str, set(size_number)))}.",
        "context": f"Thông tin về số lượng size giày hiện có.",
    }
    responses_and_context[f"Có tất cả bao nhiêu khuyến mãi", "khuyến mãi"] = {
        "response": f"Tổng số khuyến mãi hiện có là {len(set(promotion_name))}. Danh sách các khuyến mãi: {', '.join(map(str,set(promotion_name)))}.",
        "context": f"Thông tin về số lượng khuyến mãi hiện có.",
    }

    for shoe in shoes_kb:
        shoe_name = shoe.get_shoe_name()
        brand_name = shoe.get_brand_name()
        size_number = shoe.get_size_number()
        color_name = shoe.get_color_name()
        discounted_price = shoe.get_discounted_price()
        promotion_name = shoe.get_promotion_name()
        promotion_start_date = shoe.get_promotion_start_date()
        promotion_end_date = shoe.get_promotion_end_date()
        promotion_discount_percent = shoe.get_promotion_discount_percent()

        responses_and_context[f"Có {brand_name} không", f"{brand_name}"] = {
            "response": f"Có, chúng tôi có {shoe_name} từ thương hiệu {brand_name}.",
            "context": f"Thông tin về {shoe_name} từ thương hiệu {brand_name}.",
        }
        responses_and_context[
            f"Bạn có {shoe_name} không",
            f"có {shoe_name} không",
            f"{shoe_name} có không",
            f"{shoe_name}",
        ] = {
            "response": f"Có, chúng tôi có {shoe_name} với màu {color_name} và size {size_number}.",
            "context": f"Thông tin về {shoe_name} với màu {color_name} và size {size_number}.",
        }

        responses_and_context[
            f"Giày {shoe_name} màu {color_name} có đẹp không",
            f"{shoe_name} màu {color_name} đẹp không",
            f"{shoe_name} màu {color_name}",
        ] = {
            "response": f"Có, giày {shoe_name} màu {color_name} là một trong những màu được ưa chuộng.",
            "context": f"Thông tin về giày {shoe_name} màu {color_name}.",
        }

        responses_and_context[
            f"giày {shoe_name} giá bao nhiêu",
            f"{shoe_name} giá bao nhiêu",
            f"{shoe_name} giá",
        ] = {
            "response": f"Giày {shoe_name} giá {discounted_price} VNĐ.",
            "context": f"Thông tin về giày {shoe_name} giá {discounted_price} VNĐ.",
        }

        responses_and_context[
            f"khuyến mãi",
            f"khuyến mãi {promotion_name}" f"{promotion_name}",
            f"{promotion_name}",
        ] = {
            "response": f"khuyến mãi {promotion_name} bắt đầu từ ngày {promotion_start_date}.",
            "context": f"thông tin về khuyến mãi {promotion_name} bắt đầu từ ngày {promotion_start_date} kết thúc vào ngày {promotion_end_date} với giá giảm {promotion_discount_percent}%.",
        }

    return responses_and_context


# Tải dữ liệu từ cơ sở dữ liệu
shoes_kb = crud_knowledge_base.get_default_kn_chatbot(db=Session(engine), limit=25)

# Tạo phản hồi và ngữ cảnh cho từng câu hỏi
responses_and_context = create_responses_and_context(shoes_kb)

# Tạo danh sách intents từ phản hồi và ngữ cảnh đã tạo
intents = [
    {
        "tag": "greeting",
        "patterns": [
            "Xin chào",
            ".",
            "Chào shop",
            "Này",
            "Hi",
            "Xin chào",
            "Hello",
            "Alo",
        ],
        "responses": ["Xin chào, Shop có thể giúp gì cho bạn"],
        "context": [""],
    },
    {
        "tag": "goodbye",
        "patterns": [
            "Hẹn gặp lại",
            "Tạm biệt",
            "Bye",
            "Rất vui được trò chuyện với bạn, tạm biệt",
        ],
        "responses": ["Tạm biệt! Cảm ơn bạn hẹn gặp lại."],
        "context": [""],
    },
    {
        "tag": "thanks",
        "patterns": [
            "Thanks",
            "Cảm ơn",
            "Cảm ơn shop ạ",
            "Tuyệt vời, cảm ơn",
            "Cảm ơn vì đã giúp tôi",
        ],
        "responses": ["Cảm ơn bạn! Hẹn sớm gặp lại bạn"],
        "context": [""],
    },
    {
        "tag": "noanswer",
        "patterns": [],
        "responses": ["Xin vui lòng cho tôi thêm thông tin"],
        "context": [""],
    },
    {
        "tag": "options",
        "patterns": [
            "Shop có thể giúp gì",
            "Bạn có thể giúp tôi cái gì",
            "Hỗ trợ gì được cung cấp",
        ],
        "responses": [
            "Tôi có thể giúp bạn tư vấn và giới thiệu các sản phẩm mà shop có ạ!"
        ],
        "context": [""],
    },
]
for question, data in responses_and_context.items():

    intents.append(
        {
            "tag": question[0].lower().replace(" ", "_"),
            "patterns": [question],
            "responses": [data["response"]],
            "context": [data["context"]],
        }
    )

# Lưu dữ liệu đã cập nhật vào tệp intents.json
script_dir = os.getcwd()
intents_file_path = os.path.join(script_dir, "intents.json")

with open(intents_file_path, "w") as outfile:
    json.dump({"intents": intents}, outfile, ensure_ascii=False, indent=4)

In [6]:
script_dir = os.getcwd()
intents_file_path = os.path.join(script_dir, "intents.json")

with open(intents_file_path, "w") as outfile:
    json.dump({"intents": intents}, outfile, ensure_ascii=False, indent=4)